# NerF + QG Loss

The full QG equation is given by:

$$
\begin{aligned}
\partial_t q + \det \boldsymbol{J}(q, \psi) &= 0
\end{aligned}
$$

where:

* $q=\nabla^2 \psi$
* $\det \boldsymbol{J}(q, \psi)=\partial_x q\partial_y\psi - \partial_y q\partial_x\psi$.

We are interested in finding some NerF method that can take in the spatial-temporal coordinates, $\mathbf{x}_\phi$, and output a vector corresponding to the PV and stream function, $\psi$, i.e. $\mathbf{y}_\text{obs}$.

$$
\mathbf{y}_\text{obs} = \boldsymbol{f_\theta}(\mathbf{x}_\phi) + \epsilon, \hspace{5mm}\epsilon \sim \mathcal{N}(0, \sigma^2)
$$

We use a SIREN network which is a fully connected neural network with the $sin$ activation function.

* **Data Inputs**: `256x256x11`
* **Data Ouputs**: `2`


In [ ]:
import sys, os
from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".root"])

# append to path
sys.path.append(str(root))

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from ml_collections import config_dict
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger

pl.seed_everything(123)

import matplotlib.pyplot as plt
import seaborn as sns
import wandb
from config_1 import get_config


sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# from ml_collections import config_dict

# cfg = config_dict.ConfigDict()

# # logging args
# cfg.log = config_dict.ConfigDict()
# cfg.log.mode = "online" #"disabled"
# cfg.log.project ="inr4ssh"
# cfg.log.entity = "ige"
# cfg.log.log_dir = "/Users/eman/code_projects/logs/"
# cfg.log.resume = False

# # data args
# cfg.data = config_dict.ConfigDict()
# cfg.data.data_dir =  f"/Users/eman/code_projects/torchqg/data/qgsim_simple_128x128.nc"

# # preprocessing args
# cfg.pre = config_dict.ConfigDict()
# cfg.pre.noise = 0.01
# cfg.pre.dt = 1.0
# cfg.pre.time_min = 500
# cfg.pre.time_max = 511
# cfg.pre.seed = 123

# # train/test args
# cfg.split = config_dict.ConfigDict()
# cfg.split.train_prct = 0.9

# # dataloader args
# cfg.dl = config_dict.ConfigDict()
# cfg.dl.batchsize_train = 2048
# cfg.dl.batchsize_val = 1_000
# cfg.dl.batchsize_test = 5_000
# cfg.dl.batchsize_predict = 10_000
# cfg.dl.num_workers = 0
# cfg.dl.pin_memory = False

# # loss arguments
# cfg.loss = config_dict.ConfigDict()
# cfg.loss.qg = True
# cfg.loss.alpha = 1e-4

# # optimizer args
# cfg.optim = config_dict.ConfigDict()
# cfg.optim.warmup = 10
# cfg.optim.num_epochs = 100
# cfg.optim.learning_rate = 1e-4

# # trainer args
# cfg.trainer = config_dict.ConfigDict()
# cfg.trainer.accelerator = None
# cfg.trainer.devices = 1
# cfg.trainer.grad_batches = 1

In [ ]:
from inr4ssh._src.io import transform_dict

cfg = get_config()

cfg.to_dict()

In [ ]:
wandb_logger = WandbLogger(
    config=cfg.to_dict(),
    mode="offline",  # cfg.log.mode,
    project=cfg.log.project,
    entity=cfg.log.entity,
    dir=cfg.log.log_dir,
    resume=False,
)

In [ ]:
!ls /Users/eman/code_projects/torchqg/data/

## Data Module

Now we will put all of the preprocessing routines together. This is **very important** for a few reasons:

1. It collapses all of the operations in a modular way
2. It makes it reproducible for the next people
3. It makes it very easy for the PyTorch-Lightning framework down the line.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
class InputScalingTransform(nn.Module):
    def __init__(self, x_min, x_max):
        super().__init__()

        self.register_buffer("x_min", torch.FloatTensor(x_min))
        self.register_buffer("x_max", torch.FloatTensor(x_max))

    def forward(self, x, inverse=False):
        if not inverse:
            return self.transform(x)
        else:
            return self.inverse_transform(x)

    def transform(self, x):
        return (x - self.x_min) / (self.x_max - self.x_min)

    def inverse_transform(self, x):
        return x * (self.x_max - self.x_min) + self.x_min

In [ ]:
class QGSimulation(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

    def setup(self, stage=None):

        # load data
        data = xr.open_dataset(self.config.data.data_dir, engine="netcdf4")

        data_df = data.to_dataframe().reset_index()

        # subset variables of interest
        x_df = data_df[["Nx", "Ny", "steps"]]
        y_df = data_df[["p"]]

        # get spatial/temporal min/max limits
        x_min = x_df.min(axis=0)
        x_max = x_df.max(axis=0)

        # create invertible transformation
        transform = InputScalingTransform(x_min.values, x_max.values)
        self.transform = transform

        # create prediction dataset (everything)
        predict_ds = TensorDataset(
            torch.FloatTensor(x_df.values), torch.FloatTensor(y_df.values)
        )

        # create train/val/test datasets
        n_datapoints = len(predict_ds)
        train_split = int(self.config.split.train_prct * n_datapoints)
        valid_split = n_datapoints - train_split

        # random split
        train_ds, valid_ds = torch.utils.data.random_split(
            predict_ds, (train_split, valid_split)
        )

        self.ds_train = train_ds
        self.ds_valid = valid_ds
        self.ds_predict = predict_ds

    def train_dataloader(self):
        return DataLoader(
            self.ds_train,
            batch_size=self.config.dl.batchsize_train,
            shuffle=True,
            num_workers=self.config.dl.num_workers,
            pin_memory=self.config.dl.pin_memory,
        )

    def val_dataloader(self):
        return DataLoader(
            self.ds_valid,
            batch_size=self.config.dl.batchsize_val,
            shuffle=False,
            num_workers=self.config.dl.num_workers,
            pin_memory=self.config.dl.pin_memory,
        )

    def test_dataloader(self):
        return DataLoader(
            self.ds_predict,
            batch_size=self.config.dl.batchsize_test,
            shuffle=False,
            num_workers=self.config.dl.num_workers,
            pin_memory=self.config.dl.pin_memory,
        )

    def predict_dataloader(self):
        return DataLoader(
            self.ds_predict,
            batch_size=self.config.dl.batchsize_predict,
            shuffle=False,
            num_workers=self.config.dl.num_workers,
            pin_memory=self.config.dl.pin_memory,
        )

In [ ]:
dm = QGSimulation(cfg)
dm.setup()

In [ ]:
x_init, y_init = dm.ds_train[:10]

In [ ]:
x_init.shape, y_init.shape

## NerF

This standard Neural Fields.

In [ ]:
from inr4ssh._src.models.siren import Siren, SirenNet

In [ ]:
dim_in = x_init.shape[1]
dim_hidden = 256
dim_out = y_init.shape[1]
num_layers = 4
w0 = 1.0
w0_initial = 30.0
c = 6.0
final_activation = None  # nn.Sigmoid()

net = SirenNet(
    dim_in=dim_in,
    dim_hidden=dim_hidden,
    dim_out=dim_out,
    num_layers=num_layers,
    w0=w0,
    w0_initial=w0_initial,
    c=c,
    final_activation=final_activation,
)

In [ ]:
out = net(x_init)

## PINNS Loss

$$
\partial_t \nabla^2 \psi + \det J(\psi, \nabla^2\psi) = 0
$$

In [ ]:
from inr4ssh._src.operators import differential_simp as diffops_simp
import torch.nn.functional as F
from torch.optim import Adam

In [ ]:
class RegQG(nn.Module):
    def __init__(self, alpha: float = 1e-4):
        super().__init__()

        alpha = torch.Tensor([alpha])

        self.register_buffer("alpha", alpha)

    def forward(self, x, f):

        with torch.set_grad_enabled(True):

            x = torch.autograd.Variable(x, requires_grad=True)

            u = f(x)

            grad_nn = diffops_simp.gradient(u, x)
            q_nn = diffops_simp.divergence(grad_nn, x)
            dlaplacU = diffops_simp.gradient(q_nn, x)
            Jacob_U_laplacU = (
                grad_nn[:, 1] * dlaplacU[:, 2] - grad_nn[:, 2] * dlaplacU[:, 1]
            )

            pde_loss = F.mse_loss(
                dlaplacU[:, 0] + Jacob_U_laplacU, torch.zeros_like(Jacob_U_laplacU)
            )
            return self.alpha * pde_loss

In [ ]:
reg_loss = RegQG(1e-4)

In [ ]:
reg_loss(x_init, net)

## Experiment

In [ ]:
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from functools import partial
from typing import Dict, Any, cast
from torch.optim import Adam

In [ ]:
class INRModel(pl.LightningModule):
    def __init__(
        self,
        model,
        reg_pde,
        optimizer: str = "adam",
        qg: bool = True,
        alpha: float = 0.1,
        **kwargs,
    ):
        super().__init__()

        self.save_hyperparameters()
        self.model = model
        self.hyperparams = cast(Dict[str, Any], self.hparams)
        self.loss_data = nn.MSELoss(reduction="mean")
        self.reg_pde = RegQG(self.hyperparams.get("alpha", 1e-4))

    def forward(self, x):
        return self.model(x)

    def _data_loss(self, batch):
        x, y = batch

        pred = self.forward(x)

        # parse inputs
        x, y = batch

        # data loss function
        loss = self.loss_data(y, pred)

        return loss

    def _qg_loss(self, batch):

        x, y = batch

        loss = self.reg_pde.forward(x, self.model)

        return loss

    def training_step(self, batch, batch_idx):

        # loss function
        loss_data = self._data_loss(batch)

        if self.hyperparams.get("qg", False):
            # x_var = torch.autograd.Variable(x, requires_grad=True)
            # out = self.forward(x_var)
            # reg = qg_loss(out, x_var, 1.0, 1.0, 1.0, "mean")
            loss_reg = self._qg_loss(batch)

            loss = loss_data + loss_reg

            self.log("train_reg", loss_reg, prog_bar=True)
            self.log("train_data", loss_data, prog_bar=True)
        else:
            loss = loss_data

        self.log("train_loss", loss, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):

        # loss function
        loss_data = self._data_loss(batch)

        if self.hyperparams.get("qg", False):
            # x_var = torch.autograd.Variable(x, requires_grad=True)
            # out = self.forward(x_var)
            # reg = qg_loss(out, x_var, 1.0, 1.0, 1.0, "mean")
            loss_reg = self._qg_loss(batch)

            loss = loss_data + loss_reg

            self.log("train_reg", loss_reg, prog_bar=True)
            self.log("train_data", loss_data, prog_bar=True)
        else:
            loss = loss_data

        self.log("val_loss", loss, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):

        # loss function
        loss_data = self._data_loss(batch)

        if self.hyperparams.get("qg", False):
            # x_var = torch.autograd.Variable(x, requires_grad=True)
            # out = self.forward(x_var)
            # reg = qg_loss(out, x_var, 1.0, 1.0, 1.0, "mean")
            loss_reg = self._qg_loss(batch)

            loss = loss_data + loss_reg

            self.log("train_reg", loss_reg, prog_bar=True)
            self.log("train_data", loss_data, prog_bar=True)
        else:
            loss = loss_data

        self.log("test_loss", loss, prog_bar=True)

        return loss

    def predict_step(self, batch, batch_idx):
        # output
        x, y = batch

        pred = self.forward(x)

        return pred

    def configure_optimizers(self):

        # configure optimizer
        optimizer = Adam(self.model.parameters(), lr=self.hyperparams.get("lr", 1e-4))

        scheduler = LinearWarmupCosineAnnealingLR(
            optimizer,
            warmup_epochs=self.hyperparams.get("warmup", 10),
            max_epochs=self.hyperparams.get("num_epochs", 100),
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss",
        }

In [ ]:
run_path = "ige/inr4ssh/1st3rtl0"
model_path = "checkpoints/epoch=990-step=39640.ckpt"

In [ ]:
from inr4ssh._src.io import get_wandb_config, get_wandb_model

In [ ]:
best_model = get_wandb_model(run_path, model_path)
best_model.download(replace=True)

### Callbacks

In [ ]:
from pytorch_lightning.callbacks import TQDMProgressBar, ModelCheckpoint

In [ ]:
from pathlib import Path

In [ ]:
model_cb = ModelCheckpoint(
    dirpath=str(Path(wandb_logger.experiment.dir).joinpath("checkpoints")),
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

In [ ]:
callbacks = [model_cb, TQDMProgressBar(refresh_rate=1)]

In [ ]:
best_model.

### Learner

In [ ]:
state = torch.load(best_model.name, map_location=torch.device("cpu"))

In [ ]:
# state

In [ ]:
wandb_logger.experiment.

In [ ]:
# state["state_dict"]

In [ ]:
kwargs,
net = SirenNet(**kwargs)
net.load_state_dict(state_dict)

In [ ]:
from experiments.qg.qg_sim.model import INRModel

In [ ]:
learn = INRModel.load_from_checkpoint(
    best_model.name,
    model=net,
    loss_data=nn.MSELoss("mean"),
    reg_pde=reg_loss,
    learning_rate=cfg.optim.learning_rate,
    warmup=cfg.optim.warmup,
    num_epochs=cfg.optim.num_epochs,
    alpha=cfg.loss.alpha,
    qg=cfg.loss.qg,
)

### Trainer

In [ ]:
trainer = Trainer(
    min_epochs=1,
    max_epochs=cfg.optim.num_epochs,
    accelerator=cfg.trainer.accelerator,
    devices=cfg.trainer.devices,
    enable_progress_bar=True,
    logger=wandb_logger,
    callbacks=callbacks,
    accumulate_grad_batches=cfg.trainer.grad_batches,
)

### Train

In [ ]:
trainer.fit(
    learn,
    datamodule=dm,
)

## Results

### Testing

In [ ]:
# res = trainer.test(learn, dataloaders=dm.test_dataloader())

# results["data"] = res

In [ ]:
import wandb

wandb.finish()

### Predictions

In [ ]:
# t0 = time.time()
predictions = trainer.predict(learn, datamodule=dm, return_predictions=True)
predictions = torch.cat(predictions)
# t1 = time.time() - t0

In [ ]:
ds_pred = dm.create_predictions_ds(predictions)

from inr4ssh._src.operators import differential_simp as diffops_simp
from inr4ssh._src.operators import differential as diffops

In [ ]:
# ds_pred = dm.create_predictions_ds(predictions)
# ds_pred

In [ ]:
# res = trainer.test(learn, datamodule=dm)
# t0 = time.time()
predictions = trainer.predict(learn, datamodule=dm, return_predictions=True)
predictions = torch.cat(predictions)

ds_pred = dm.create_predictions_ds(predictions)

from inr4ssh._src.operators import differential_simp as diffops_simp
from inr4ssh._src.operators import differential as diffops

learn.model.eval()
coords, truths, preds, grads, qs = [], [], [], [], []
for ix, iy in tqdm.tqdm(dm.predict_dataloader()):
    with torch.set_grad_enabled(True):
        # prediction
        ix = torch.autograd.Variable(ix.clone(), requires_grad=True)
        p_pred = learn.model(ix)

        # p_pred = p_pred.clone()
        # p_pred.require_grad_ = True

        # gradient
        p_grad = diffops_simp.gradient(p_pred, ix)
        # p_grad = diffops.grad(p_pred, ix)
        # q
        q = diffops_simp.divergence(p_grad, ix)
        # q = diffops.div(p_grad, ix)

    # collect
    truths.append(iy)
    coords.append(ix)
    preds.append(p_pred)
    grads.append(p_grad)
    qs.append(q)

coords = torch.cat(coords).detach().numpy()
preds = torch.cat(preds).detach().numpy()
truths = torch.cat(truths).detach().numpy()
grads = torch.cat(grads).detach().numpy()
qs = torch.cat(qs).detach().numpy()

### Figure I: Predictions

In [ ]:
ds_pred.pred.thin(time=1).plot.imshow(
    col="time",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

In [ ]:
ds_pred.pred.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### Figure II: Ground Truth

In [ ]:
ds_pred.true.thin(time=1).plot.imshow(
    col="time",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

In [ ]:
ds_pred.true.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### Figure III: Absolute Error

In [ ]:
(ds_pred.true - ds_pred.pred).thin(time=1).plot.imshow(
    col="time",
    robust=True,
    col_wrap=3,
    cmap="RdBu_r",
)

In [ ]:
(ds_pred.true - ds_pred.pred).hvplot.image(
    x="Nx", y="Ny", width=500, height=400, cmap="viridis"
)

# PINN

In [ ]:
def laplace(y, x):
    grad = gradient(y, x)
    return divergence(grad, x)


def divergence(y, x):
    div = 0.0
    for i in range(y.shape[-1]):
        div += torch.autograd.grad(
            y[..., i], x, torch.ones_like(y[..., i]), create_graph=True
        )[0][..., i : i + 1]
    return div


def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [ ]:
ydata = data.p.values[:nbmax, :ss, :ss].flatten()[:, None]
# ydataT = torch.from_numpy(ydata)#.float()

In [ ]:
traindata.shape, ydata.shape

In [ ]:
mseloss = torch.nn.MSELoss()


def PINNloss(pred_p, lab, coords, w=1e-4):  # 1e-4
    grad_nn = gradient(pred_p, coords)
    q_nn = divergence(grad_nn, coords)
    dlaplacU = gradient(q_nn, coords)
    Jacob_U_laplacU = grad_nn[:, 1] * dlaplacU[:, 2] - grad_nn[:, 2] * dlaplacU[:, 1]
    ###########
    function_loss = mseloss(dlaplacU[:, 0] + Jacob_U_laplacU, torch.zeros(1).double())
    data_loss = mseloss(pred_p, lab)
    return data_loss + w * function_loss

In [ ]:
perm = np.random.permutation(len(traindata))
nbsample = np.int64(len(traindata) * 0.1)  # 1% of data
nbsample

In [ ]:
training_loader = torch.utils.data.DataLoader(
    list(zip(traindata[perm, :][:nbsample, :], ydata[perm, :][:nbsample, :])),
    batch_size=32,
    shuffle=True,
)

In [ ]:
# Model instantiation
torch.manual_seed(2022)

model = Siren(
    in_features=3,
    out_features=1,
    hidden_features=64,
    hidden_layers=3,
    outermost_linear=True,
).double()
# model.load_state_dict(torch.load('/home/rlguensat/MLstuff/SIREN_addPhysLoss_SGD.h5'))#_finetune_addPhysLoss

In [ ]:
optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())


def train_one_epoch():
    running_loss = 0.0
    last_loss = 0.0
    epoch_loss = 0.0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs, coords = model(inputs)

        # Compute the loss and its gradients
        loss = PINNloss(outputs, labels, coords)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # epoch loss
        epoch_loss += outputs.shape[0] * loss.item()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000  # print every 1000 mini-batch
            print("  batch {} loss: {}".format(i + 1, last_loss))
            running_loss = 0.0

    return epoch_loss

In [ ]:
h_adam = []

EPOCHS = 100

model.train()

for epoch in range(EPOCHS):
    print("EPOCH {}:".format(epoch + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    epoch_loss = train_one_epoch()
    h_adam.append(epoch_loss)
    print("EPOCH loss {}:".format(epoch_loss))

In [ ]:
# 16
plt.semilogy(h_adam, label="Adam")
plt.legend()

In [ ]:
# torch.save(model.state_dict(), '/home/rlguensat/MLstuff/SIREN_addPhysLoss_noweighting.h5')

# Let us take the first image

In [ ]:
point = torch.from_numpy(traindata[: ss * 256, :])
point.shape

In [ ]:
point

In [ ]:
model.eval()
pred_p, cc = model(point)
grad_nn = gradient(pred_p, cc)

In [ ]:
q_nn = divergence(grad_nn, cc)

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.imshow(pred_p.detach().numpy().reshape((256, ss)))
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(data.p[0, :256, :ss])
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(
    pred_p.detach().numpy().reshape((256, ss)) - data.p[0, :256, :ss], cmap="coolwarm"
)
plt.colorbar()

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.imshow((grad_nn[:, 1]).detach().numpy().reshape((256, ss)))
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(data.v[0, :256, :ss])
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(
    (grad_nn[:, 1]).detach().numpy().reshape((256, ss)) - data.v[0, :256, :ss],
    cmap="coolwarm",
)
plt.colorbar()

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.imshow((q_nn[:, 0]).detach().numpy().reshape((256, ss)), vmin=-2, vmax=2)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(data.q[0, :256, :ss], vmin=-2, vmax=2)
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(
    (q_nn[:, 0]).detach().numpy().reshape((256, ss)) - data.q[0, :256, :ss],
    cmap="coolwarm",
)
plt.colorbar()

# Equation

$$  \partial_t q + \boldsymbol{J}(p, q)= 0  $$

In [ ]:
dlaplacU = gradient(q_nn, cc)
Jacob_U_laplacU = grad_nn[:, 1] * dlaplacU[:, 2] - grad_nn[:, 2] * dlaplacU[:, 1]

In [ ]:
plt.hist((dlaplacU[:, 0] + Jacob_U_laplacU).detach().numpy(), bins=100)